In [ ]:
#| default_exp utils.data
# | hide
import nbdev
from nbdev.showdoc import *

nbdev.nbdev_export()


In [ ]:
#| export
from tidygrad.tensor import Tensor
import os
import requests


In [ ]:
#| export
def download_file(url, path):
    dir_path = os.path.dirname(path)

    os.makedirs(dir_path, exist_ok=True)
    if not os.path.exists(path):
        response = requests.get(url)
        with open(path, "wb") as f:
            f.write(response.content)

In [ ]:
# | export
### Fastai style
class Dataset:
    def __getitem__(self, idx):
        raise NotImplementedError

    def __len__(self):
        raise NotImplementedError

    def shuffle(self):
        raise NotImplementedError

    def collate_fn(self, batch):
        raise NotImplementedError

class DataLoader:
    def __init__(self, dataset, batch_size=64, shuffle=True, batch_tfms=()):
        self.batch_tfms = batch_tfms
        self.dataset = dataset  # indexed dataset
        self.batch_size = batch_size
        self.shuffle = shuffle

    def __len__(self):
        return len(self.dataset) // self.batch_size

    def __iter__(self):
        if self.shuffle:
            self.dataset.shuffle()
        self.i = 0
        return self

    def __next__(self):
        if self.i + self.batch_size > len(self.dataset):
            raise StopIteration

        batch = [self.dataset[self.i + i] for i in range(self.batch_size)]
        self.i += self.batch_size
        collated = self.dataset.collate_fn(batch)
        for tfm in self.batch_tfms:
            collated = tfm(*collated)
        return [Tensor(t) for t in collated]

class DataLoaders:
    def __init__(self, train: DataLoader, test: DataLoader):
        self.train = train
        self.test = test